# Notebook ICD - 7

Install libraries via terminal

In [ ]:
!pip install spacy
!pip install nltk
!pip install sklearn

  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


Importing necessary functions

In [ ]:
import pandas as pd
import re
import spacy as spc
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer

## Datos de entrada

In [ ]:
oracion = "@Mar18blink Estoy buscando un producto genial por menos de 50. He encontrado muchos en https://t.co/FI1ShlDozc. ¿Qué opinas? Quiero comprarlo ya!!!"
print(oracion)

@Mar18blink Estoy buscando un producto genial por menos de 50. He encontrado muchos en https://t.co/FI1ShlDozc. ¿Qué opinas? Quiero comprarlo ya!!!


## Procesamiento

### Limpieza de datos

In [ ]:
oracion = oracion.lower()
oracion = re.sub(r"@\S+", "", oracion)  # Eliminar menciones a usuarios
oracion = re.sub("http[s]?\://\S+", "", oracion)  # Eliminar enlaces
oracion = re.sub(r"#\S+", "", oracion)  # Eliminar hashtags
oracion = re.sub(r"[0-9]", "", oracion)  # Eliminar números
oracion = re.sub(r"(\(.*\))|(\[.*\])", "", oracion)  # Eliminar paréntesis y corchetes
oracion = re.sub(r"\n", "", oracion)  # Eliminar caracteres de nueva línea
oracion = re.sub(r"(http[s]?\://\S+)|([\[\(].*[\)\]])|([#@]\S+)|\n", "", oracion)  # Eliminar varios patrones
oracion = re.sub(r"(\.)|(,)", "", oracion)  # Eliminar puntos y comas
oracion = re.sub(r"[¡!]", "", oracion)  # Eliminar signos de admiración
oracion = re.sub(r"[¿?]", "", oracion)  # Eliminar signos de exclamación
print(oracion)

 estoy buscando un producto genial por menos de  he encontrado muchos en  qué opinas quiero comprarlo ya


### Tokenización

In [ ]:
nltk.download('punkt')
tokens = word_tokenize(oracion)
print(tokens)

['estoy', 'buscando', 'un', 'producto', 'genial', 'por', 'menos', 'de', 'he', 'encontrado', 'muchos', 'en', 'qué', 'opinas', 'quiero', 'comprarlo', 'ya']


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### Stopwords

In [ ]:
nltk.download('stopwords')
spanish_stopwords = stopwords.words('spanish')
palabras_filtradas = [palabra for palabra in tokens if palabra not in spanish_stopwords]
print(palabras_filtradas)

['buscando', 'producto', 'genial', 'menos', 'encontrado', 'opinas', 'quiero', 'comprarlo']


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Lematización

In [ ]:
!sudo python3 -m spacy download es

⚠ As of spaCy v3.0, shortcuts like 'es' are deprecated. Please use the
full pipeline package name 'es_core_news_sm' instead.
  Using cached https://github.com/explosion/spacy-models/releases/download/es_core_news_sm-3.7.0/es_core_news_sm-3.7.0-py3-none-any.whl (12.9 MB)
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
nlp = spc.load("es_core_news_sm")
lema = nlp(" ".join(palabras_filtradas))
oracion_lematizada = " ".join([token.lemma_ for token in lema])
print(oracion_lematizada)

buscar producto genial menos encontrado opina querer comprar él


## Bag of Words

In [ ]:
vectorizador = CountVectorizer()
vectores = vectorizador.fit_transform([oracion_lematizada])
vocabulario = vectorizador.get_feature_names_out()

### Resultados

In [ ]:
print("Oración de entrada:", oracion)
print("Oración lematizada:", oracion_lematizada)
print("Vectores Bag of Words:", vectores.toarray())
print("Vocabulario:", vocabulario)

Oración de entrada:  estoy buscando un producto genial por menos de  he encontrado muchos en  qué opinas quiero comprarlo ya
Oración lematizada: buscar producto genial menos encontrado opina querer comprar él
Vectores Bag of Words: [[1 1 1 1 1 1 1 1 1]]
Vocabulario: ['buscar' 'comprar' 'encontrado' 'genial' 'menos' 'opina' 'producto'
 'querer' 'él']


### Construir Dataframe

In [ ]:
df_bw = pd.DataFrame.sparse.from_spmatrix(vectores,columns = vocabulario)
df_bw.head()

,buscar,comprar,encontrado,genial,menos,opina,producto,querer,él
0,1,1,1,1,1,1,1,1,1
